In [1]:
import torch as pt
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.utils import save_image

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5)),
])

In [3]:
device = "cuda" if pt.cuda.is_available() else "cpu"
latent_dim = 100
batch_size = 64
gen_lr = 2e-4
disc_lr = 2e-4
epochs = 50
stats = (0.5), (0.5)

In [4]:
train_dataset = MNIST("./data", train=True, download=True, transform=transforms_)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = MNIST("./data", train=False, download=True, transform=transforms_)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
# def denorm(img_tensors):
#     return img_tensors * stats[1][0] + stats[0][0]

In [6]:
def show_images(images, n_max=batch_size):
    _, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    # ax.imshow(make_grid(denorm(images.detach().cpu()[:n_max]), nrow=8).permute(1, 2, 0))
    ax.imshow(images.detach().cpu()[:n_max]).permute(1, 2, 0)

In [7]:
from torch import nn
from torch.nn import functional as F

In [8]:
class Generator(nn.Module):
    def __init__(self, latent_dim, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.conv_transpose_block1 = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, kernel_size=8, padding=0, stride=2, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True), # (64 (batch_size), 256 (output_shape), 4 (kernel_size), 4 (kernel_size))

            nn.ConvTranspose2d(512, 256, kernel_size=6, padding=1, stride=2, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True), # (64, 256, 14, 14) (..., ..., (prev kernel total) + (current kernel), (prev kernel total) + (current kernel))

            nn.ConvTranspose2d(256, 128, kernel_size=4, padding=1, stride=2, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True), # (64, 128, 19, 19)

            nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1, stride=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True), # (64, 64, 22, 22)

            # nn.ConvTranspose2d(64, 32, kernel_size=4, padding=1, stride=2, bias=True),
            # nn.BatchNorm2d(32),
            # nn.ReLU(True), # (64, 32, 26, 26)

            # nn.ConvTranspose2d(32, 28, kernel_size=2, padding=1, stride=2, bias=True),
            # nn.BatchNorm2d(28),
            # nn.ReLU(True), # (64, 28, 28, 28)

            # nn.Conv2d(32, 1, kernel_size=4, padding=1, stride=2, bias=True),
            # ((6 - 4) + (2*1) / 2) + 1 = 3
        )

    def forward(self, x):
        x = self.conv_transpose_block1(x)
        print(f"x.size(): {x.size()}")
        return F.tanh(x)

In [9]:
generator = Generator(latent_dim).to(device)

xb = pt.randn(batch_size, latent_dim, 1, 1) # random latent tensors - (columns (width), rows (height), 1, 1)
# print(f"xb.size(): {xb.size()}")
fake_images = generator(xb.to(device))
# print(fake_images.shape)
# show_images(fake_images)

x.size(): torch.Size([64, 64, 36, 36])


/home/parth/miniconda3/envs/env-pytorch/lib/python3.8/site-packages/torch/nn/modules/conv.py:956: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343998658/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv_transpose2d(


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.discriminator = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.LeakyReLU(0.2, inplace=True),
            # (28 – 3 + (2*1)) / 1 + 1 = 28 / maxpool = 14

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.LeakyReLU(0.2, inplace=True),
            # (14 – 3 + (2*1)) / 1 + 1 = 14 / maxpool = 7

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.LeakyReLU(0.2, inplace=True),
            # (7 – 3 + (2*1)) / 1 + 1 = 7 / maxpool = 3
        )
        self.dense1 = nn.Linear(256*3*3, 1)
        # self.dropout = nn.Dropout(0.5)
        # self.dense2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.discriminator(x)
        x = x.view(-1, 256*4*4)
        x = self.dense1(x)
        # x = self.dropout(x)
        # x = self.dense2(x)
        return F.sigmoid(x)

In [ ]:
discriminator = Discriminator().to(device)

In [ ]:
def train_generator(opt_g):
    # Clear generator gradients
    opt_g.zero_grad()

    # Generate fake images
    latent = pt.randn(batch_size, latent_dim, 1, 1, device=device)
    fake_images = generator.train()(latent)

    # Try to fool the discriminator
    preds = discriminator(fake_images)
    targets = pt.ones(batch_size, 1, device=device)
    loss = F.binary_cross_entropy(preds, targets)

    # Update generator weights
    loss.backward()
    opt_g.step()

    return loss.item()

In [ ]:
def train_discriminator(real_images, opt_d):
    # Clear discriminator gradients
    opt_d.zero_grad()

    # Pass real images through discriminator
    real_preds = discriminator.train()(real_images)
    real_targets = pt.ones(real_images.size(0), 1, device=device)
    real_loss = F.binary_cross_entropy(real_preds, real_targets)
    # real_score = pt.mean(real_preds).item()

    # Generate fake images
    latent = pt.randn(batch_size, latent_dim, 1, 1, device=device)
    fake_images = generator.train()(latent)

    # Pass fake images through discriminator
    fake_targets = pt.zeros(fake_images.size(0), 1, device=device)
    fake_preds = discriminator.train()(fake_images)
    fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)
    # fake_score = pt.mean(fake_preds).item()

    # Update discriminator weights
    loss = real_loss + fake_loss # (... + ...) / 2
    loss.backward()
    opt_d.step()
    return loss.item(), real_loss.detach().cpu().item(), fake_loss.detach().cpu().item() # real_score, fake_score

In [ ]:
import os

sample_dir = 'generated-images'
os.makedirs(sample_dir, exist_ok=True)

In [ ]:
def save_samples(index, latent_tensors, show=True):
    fake_images = generator.eval()(latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(
        # denorm(fake_images.detach().cpu()),
        fake_images.detach().cpu(),
        os.path.join(sample_dir, fake_fname),
        nrow=8
    )
    print('Saving', fake_fname)
    if show:
        _, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.detach().cpu(), nrow=8).permute(1, 2, 0))
        plt.show()

In [ ]:
opt_g = pt.optim.Adam(generator.parameters(), lr=gen_lr, betas=(0.5, 0.999))
opt_d = pt.optim.Adam(discriminator.parameters(), lr=disc_lr, betas=(0.5, 0.999))

fixed_latent = pt.randn(64, latent_dim, 1, 1, device=device)

In [ ]:
import numpy as np
from tqdm import tqdm

In [ ]:
losses_g, losses_d = [], []
real_scores, fake_scores = [], []

for epoch in range(epochs):
    batch_loss_g, batch_loss_d = [], []
    batch_real_score, batch_fake_score = [], []

    for loader in train_loader, test_loader:
        # Train discriminator
        for real_images, _ in tqdm(loader):
            loss_d, real_score, fake_score = train_discriminator(real_images.to(device), opt_d)
            batch_loss_d.append(loss_d)
            batch_real_score.append(real_score)
            batch_fake_score.append(fake_score)
            # Train generator
            loss_g = train_generator(opt_g)
            batch_loss_g.append(loss_g)

    # Record losses & scores
    losses_g.append(np.mean(batch_loss_g))
    losses_d.append(np.mean(batch_loss_d))
    real_scores.append(np.mean(batch_real_score))
    fake_scores.append(np.mean(batch_fake_score))

    batch_loss_g, batch_loss_d = [], []
    batch_real_score, batch_fake_score = [], []

    # Save generated images
    save_samples(epoch+1, fixed_latent, show=False)

    # Log losses & scores (last batch)
    print(f"Epoch [{epoch+1}/{epochs}], loss_g: {loss_g:.4f}, loss_d: {loss_d:.4f},"
          + f" real_score: {real_score:.4f}, fake_score: {fake_score:.4f}\n")

In [ ]:
plt.figure(figsize=(10, 8), dpi=100)

plt.subplot(2, 2, 1)
plt.plot(losses_g, '-')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Generator Loss')

plt.subplot(2, 2, 2)
plt.plot(losses_d, '-')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Discriminator Loss')

plt.subplot(2, 2, 3)
plt.plot(real_scores, '-')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Real Images Discriminator Scores')

plt.subplot(2, 2, 4)
plt.plot(fake_scores, '-')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Fake Images Discriminator Scores')

In [ ]:
pt.save(generator.state_dict(), "models/generator-model")
pt.save(discriminator.state_dict(), "models/discriminator-model")